In [ ]:
#Algorithm
#read all seeds of particular type
#read file contents in a string
#get all locations of each seed in list
#get all left and right contexts of each seed in list



#remove all newlines from text
#get k particular characters from left and right
#get theier prefixes in list
#get all prefixes
#find their prefixes again
#pick all prefixes and if exist in each seed so do a count

In [6]:
seeds = []
seedsLocation = "data/seeds2"
with open(seedsLocation) as f:
    seeds = [line.strip() for line in f]
print("lines are: ")
print(seeds)

lines are: 
['Android', '2GB', '150 g']


In [8]:
def common_count(t0, t1):
  "returns the length of the longest common prefix"
  for i, pair in enumerate(zip(t0, t1)):
    if pair[0] != pair[1]:
      return i
  return i

def group_by_longest_prefix(iterable):
  "given a sorted list of strings, group by longest common prefix"
  longest = 0
  out = []

  for t in iterable:
    if out: # if there are previous entries 

      # determine length of prefix in common with previous line
      common = common_count(t, out[-1])

      # if the current entry has a shorted prefix, output previous 
      # entries as a group then start a new group
      if common < longest:
        yield out
        longest = 0
        out = []
      # otherwise, just update the target prefix length
      else:
        longest = common

    # add the current entry to the group
    out.append(t)

  # return remaining entries as the last group
  if out:
    yield out

In [9]:
def createAllPrefix (s, l):
    ansList = []
    if len(l)==1:
        for ch in l[0]:
            s = s+ch 
            ansList.append(s.strip())
        return list(set(ansList))
    dictMap = {}
    for item in l:
        ch = item[0]
        if ch in dictMap:
            dictMap[ch].append(item[1:])
        else:
            dictMap[ch] = [item[1:]]
    for k, v in dictMap.items():
        ansList.append((s+k).strip())
        nextAnswer = createAllPrefix(s+k, v)
        ansList.extend(nextAnswer)
    return list(set(ansList))
    
    
def createPossibleStrings(l):
    return set(createAllPrefix ("", l))

def getCommonPrefix(l):
    if len(l)==1:
        return l
    for item in l:
        if len(l)==0:
            return [""]
    dictMap = {}
    for item in l:
        ch=item[0]
        if ch in dictMap:
            dictMap[ch].append(item[1:])
        else:
            dictMap[ch] = [item[1:]]
    ansList = []
    for k,v in dictMap.items():
        nextAnswer = getCommonPrefix(v)
        if len(nextAnswer) == 1:
            ansList.append(k + nextAnswer[0])
        else:
            ansList.append(k+"")
    return ansList
        
    
def removeDuplicates(l):
    if len(l)==1:
        return l
    i=l[0]
    rest=l[1:]
    ans = []
    for item in rest:
        if item.startswith(i):
            return removeDuplicates(rest)
    ans.append(i)
    ans.extend(removeDuplicates(rest))
    return ans

# getCommonPrefix(["sanjeev", "kumar", "sanju","sani", "sanji"])
createAllPrefix("", ["sanjeev", "kumar"])

['kumar',
 'sanjee',
 'san',
 'sanje',
 'k',
 'kum',
 's',
 'kuma',
 'sanjeev',
 'ku',
 'sa',
 'sanj']

In [200]:
text = ["sanjeeev", "sanjeev"]
result = group_by_longest_prefix(text)
for ans in result:
    print ans

['sanjeeev', 'sanjeev']


In [93]:
"sanjeev\n\nkumar".strip()

'sanjeev\n\nkumar'

In [10]:
def doIntersection(l):
    if len(l)==1:
        return []
    s1 = l[0]
    s2 = l[1:]
    result = []
    for s in s2:
        r = s1.intersection(s)
        result.extend(r)
    result.extend(doIntersection(s2))
    return list(set(result))

l = [set([1,2]), set([2]), set([2,3, 5]), set([4,5])]
doIntersection(l)

[2, 5]

In [11]:
import os
import re
ecommerceFile = "data/Lenovo Vibe K5.html"
ecommerceData = ""
with open(ecommerceFile) as f:
    ecommerceData = f.read()
ecommerceData = re.sub( '\s+', ' ', ecommerceData).strip()


presentSeeds = []





def getIsAnyPatternInsideFlag(result, myL):
    for item in myL:
        if item in result:
            return True
    return False


def isTag(s):
    if s.find("</")!=-1 and s.find(">")!=-1:
        return True

for seed in seeds:
    seedLocInPage = ecommerceData.find(seed)
    if seedLocInPage == -1:
        continue
    else:
        presentSeeds.append(seed)

        
print("Presnet seeds are ")
print(presentSeeds)
leftContexts = []
rightContexts = []

for seed in presentSeeds:
    lcPerSeed = []
    rcPerSeed = []
    for m in re.finditer(seed, ecommerceData):
        start = m.start()
        end   = m.end()
        leftContextPerFinding = ecommerceData[:start].strip()[::-1]
        if len(leftContextPerFinding)>100:
            leftContextPerFinding = leftContextPerFinding[:100].strip()
        lcPerSeed.append(leftContextPerFinding)
        rightContextPerFinding = ecommerceData[end:].strip()
        if len(rightContextPerFinding)>100:
            rightContextPerFinding = rightContextPerFinding[:100].strip()
        rcPerSeed.append(rightContextPerFinding)
    leftContexts.append(createPossibleStrings(lcPerSeed))
    rightContexts.append(createPossibleStrings(rcPerSeed))

# print(leftContexts)
# print(rightContexts)
myLPatterns = removeDuplicates(doIntersection(leftContexts))
myRPatterns = removeDuplicates(doIntersection(rightContexts))
    
revLPatterns = []
for p in myLPatterns:
    revLPatterns.append(p[::-1])
print("myLPatterns")
print(revLPatterns)
print("MyRPatterns")
print(myRPatterns)
results = []
for lPattern in revLPatterns:
    leftPatternLength = len(lPattern)
    if leftPatternLength<=0:
        continue
    for rPattern in myRPatterns:
        rightPatternLength = len(rPattern)
        if rightPatternLength<=0:
            continue
        pos = 0
        wholePage = ecommerceData
        while True:
            pos=wholePage.find(lPattern)
            if pos==-1:
                break
            wholePage = wholePage[pos+leftPatternLength:]
            pos = wholePage.find(rPattern)
            if(pos==-1):
                break
            result = wholePage[:pos].strip()
            insideFlag = getIsAnyPatternInsideFlag(result, myL)
            if insideFlag==False:
                results.append(result)
#                 print("result is " + result)
                wholePage = wholePage[pos:]
    
results = list(set(results))
print("Results are\n===========================")
for item in results:
    if isTag(item):
        continue
    print(item)
    
    
#                 print("fist chars are " + wholePage[:10])
#             else:
                
#             print("Result is ")
#             print(result)
#             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
#                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]

    
    
    
    
# print("Category classification")
# for c in leftContexts:
#     print(c)
    
# print("Right category classification")
# for c in rightContexts:
#     print(c)
# flatLeftContexts = list(set([item for sublist in leftContexts for item in sublist]))
# flatRightContexts = list(set([item for sublist in rightContexts for item in sublist]))

# flatLeftContexts = getAllPrefixes(flatLeftContexts)
# flatRightContexts = getAllPrefixes(flatRightContexts)
# print("Left context are ")
# print(flatLeftContexts)
# print("right context are")
# print(flatRightContexts)


# leftContextDict = {}
# for item in flatLeftContexts:
#     leftContextDict[item] = 0
# rightContextDict = {}
# for item in flatRightContexts:
#     rightContextDict[item]= 0
    
    
    
# for perSeedLeft in leftContexts:
#     for seed in perSeedLeft:
#         for prefix in flatLeftContexts:
#             if seed.startswith(prefix):
#                 leftContextDict[prefix] = leftContextDict[prefix] + 1
# for perSeedRight in rightContexts:
#     for seed in perSeedRight:
#         for prefix in flatRightContexts:
#             if seed.startswith(prefix):
#                 rightContextDict[prefix] = rightContextDict[prefix] + 1

                
# myLeftPatterns = []
# for key, value in leftContextDict.items():
#     if value>1:
#         myLeftPatterns.append(key)


# myRightPatterns = []
# for key, value in rightContextDict.items():
#     if value>1:
#         myRightPatterns.append(key)

        
# print("Left contexts:")
# print(leftContextDict)
# print("right contexts:")
# print(rightContextDict)


# print("My Left")
# print(myLeftPatterns)        
# print("My right")
# print(myRightPatterns)

# myL = []
# #reverse all patterns now
# for item in myLeftPatterns:
#     myL.append(item[::-1])
# flatL = []
# for item in flatLeftContexts:
#     flatL.append(item[::-1])
    
    
# print("My L")
# print(myL)
# print("flat l")
# print(flatL)


# results = []
# for lPattern in myL:
#     leftPatternLength = len(lPattern)
#     if leftPatternLength<=0:
#         continue
#     for rPattern in flatRightContexts:
#         rightPatternLength = len(rPattern)
#         if rightPatternLength<=0:
#             continue
#         pos = 0
#         wholePage = ecommerceData
#         while True:
#             pos=wholePage.find(lPattern)
#             if pos==-1:
#                 break
#             wholePage = wholePage[pos+leftPatternLength:]
#             pos = wholePage.find(rPattern)
#             if(pos==-1):
#                 break
#             result = wholePage[:pos].strip()
#             insideFlag = getIsAnyPatternInsideFlag(result, myL)
#             if insideFlag==False:
#                 results.append(result)
#                 print("result is " + result)
#                 wholePage = wholePage[pos:]
#                 print("fist chars are " + wholePage[:10])
#             else:
                
#             print("Result is ")
#             print(result)
#             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
#                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]

    
# for rPattern in myRightPatterns:
#     rightPatternLength = len(rPattern)
#     if rightPatternLength<=0:
#         continue
#     for lPattern in flatLeftContexts:
#         leftPatternLength = len(lPattern)
#         if leftPatternLength<=0:
#             continue
#         pos = 0
#         wholePage = ecommerceData
#         while True:
#             pos=wholePage.find(lPattern)
#             if pos==-1:
#                 break
#             wholePage = wholePage[pos+leftPatternLength:]
#             pos = wholePage.find(rPattern)
#             if(pos==-1):
#                 break
#             result = wholePage[:pos].strip()
#             insideFlag = getIsAnyPatternInsideFlag(result, flatLeftContexts)
#             if insideFlag==False:
#                 results.append(result)
#                 wholePage = wholePage[pos+rightPatternLength:]
#                 print("fist chars are " + wholePage[:10])
#             print("Result is ")
#             print(result)
# #             results.append(result)
#             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
#                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]
# print("All results are")
# for result in results:
#     print(result)
# print(results)

Presnet seeds are 
['Android', '2GB', '150 g']
myLPatterns
['</td><td class="value">', 'd><td class="value">', 'class="value">', 'lue">']
MyRPatterns
['</td></tr> <tr', '</td></tr> <t', '</td></t', '</td>', '<']


NameError: name 'myL' is not defined

In [12]:
import os
import re
ecommerceFile = "data/Lenovo Vibe K5.html"
ecommerceData = ""
with open(ecommerceFile) as f:
    ecommerceData = f.read()
ecommerceData = re.sub( '\s+', ' ', ecommerceData).strip()


presentSeeds = []


def getIsAnyPatternInsideFlag(result, myL):
    for item in myL:
        if item in result:
            return True
    return False

for seed in seeds:
    seedLocInPage = ecommerceData.find(seed)
    if seedLocInPage == -1:
        continue
    else:
        presentSeeds.append(seed)

        
print("Presnet seeds are ")
print(presentSeeds)
leftContexts = []
rightContexts = []

for seed in presentSeeds:
    lcPerSeed = []
    rcPerSeed = []
    for m in re.finditer(seed, ecommerceData):
        start = m.start()
        end   = m.end()
        leftContextPerFinding = ecommerceData[:start].strip()[::-1]
        if len(leftContextPerFinding)>100:
            leftContextPerFinding = leftContextPerFinding[:100]
        lcPerSeed.append(leftContextPerFinding)
        rightContextPerFinding = ecommerceData[end:].strip()
        if len(rightContextPerFinding)>100:
            rightContextPerFinding = rightContextPerFinding[:100]
        rcPerSeed.append(rightContextPerFinding)
    print("Seed is " + seed)
    print("rc persed is ")
    print(rcPerSeed)
    lcPerSeed = getAllPrefixes(lcPerSeed)
    rcPerSeed = getAllPrefixes(rcPerSeed)
    leftContexts.append(lcPerSeed)
    rightContexts.append(rcPerSeed)

# print("Category classification")
# for c in leftContexts:
#     print(c)
    
# print("Right category classification")
# for c in rightContexts:
#     print(c)
flatLeftContexts = list(set([item for sublist in leftContexts for item in sublist]))
flatRightContexts = list(set([item for sublist in rightContexts for item in sublist]))

flatLeftContexts = getAllPrefixes(flatLeftContexts)
flatRightContexts = getAllPrefixes(flatRightContexts)
print("Left context are ")
print(flatLeftContexts)
print("right context are")
print(flatRightContexts)


leftContextDict = {}
for item in flatLeftContexts:
    leftContextDict[item] = 0
rightContextDict = {}
for item in flatRightContexts:
    rightContextDict[item]= 0
    
    
    
for perSeedLeft in leftContexts:
    for seed in perSeedLeft:
        for prefix in flatLeftContexts:
            if seed.startswith(prefix):
                leftContextDict[prefix] = leftContextDict[prefix] + 1
for perSeedRight in rightContexts:
    for seed in perSeedRight:
        for prefix in flatRightContexts:
            if seed.startswith(prefix):
                rightContextDict[prefix] = rightContextDict[prefix] + 1

                
myLeftPatterns = []
for key, value in leftContextDict.items():
    if value>1:
        myLeftPatterns.append(key)


myRightPatterns = []
for key, value in rightContextDict.items():
    if value>1:
        myRightPatterns.append(key)

        
print("Left contexts:")
print(leftContextDict)
print("right contexts:")
print(rightContextDict)


print("My Left")
print(myLeftPatterns)        
print("My right")
print(myRightPatterns)

myL = []
#reverse all patterns now
for item in myLeftPatterns:
    myL.append(item[::-1])
flatL = []
for item in flatLeftContexts:
    flatL.append(item[::-1])
    
    
print("My L")
print(myL)
print("flat l")
print(flatL)


results = []
for lPattern in myL:
    leftPatternLength = len(lPattern)
    if leftPatternLength<=0:
        continue
    for rPattern in flatRightContexts:
        rightPatternLength = len(rPattern)
        if rightPatternLength<=0:
            continue
        pos = 0
        wholePage = ecommerceData
        while True:
            pos=wholePage.find(lPattern)
            if pos==-1:
                break
            wholePage = wholePage[pos+leftPatternLength:]
            pos = wholePage.find(rPattern)
            if(pos==-1):
                break
            result = wholePage[:pos].strip()
            insideFlag = getIsAnyPatternInsideFlag(result, myL)
            if insideFlag==False:
                results.append(result)
                print("result is " + result)
                wholePage = wholePage[pos:]
                print("fist chars are " + wholePage[:10])
#             else:
                
#             print("Result is ")
#             print(result)
#             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
#                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]

    
# for rPattern in myRightPatterns:
#     rightPatternLength = len(rPattern)
#     if rightPatternLength<=0:
#         continue
#     for lPattern in flatLeftContexts:
#         leftPatternLength = len(lPattern)
#         if leftPatternLength<=0:
#             continue
#         pos = 0
#         wholePage = ecommerceData
#         while True:
#             pos=wholePage.find(lPattern)
#             if pos==-1:
#                 break
#             wholePage = wholePage[pos+leftPatternLength:]
#             pos = wholePage.find(rPattern)
#             if(pos==-1):
#                 break
#             result = wholePage[:pos].strip()
#             insideFlag = getIsAnyPatternInsideFlag(result, flatLeftContexts)
#             if insideFlag==False:
#                 results.append(result)
#                 wholePage = wholePage[pos+rightPatternLength:]
#                 print("fist chars are " + wholePage[:10])
#             print("Result is ")
#             print(result)
# #             results.append(result)
#             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
#                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]
print("All results are")
for result in results:
    print(result)
# print(results)

Presnet seeds are 
['Android', '2GB', '150 g']
Seed is Android
rc persed is 
['v5.1 Lollipop operating system with 1.5GHz Qualcomm Snapdragon 415 octa core processor, Adreno 405 G', '</td></tr> <tr><td class="label"> RAM </td><td class="value">2 GB</td></tr> <tr class="size-weight">', '5version, no sensors.</div><div class="a-expander-header a-expander-partial-collapse-header" style="']


NameError: name 'getAllPrefixes' is not defined

In [89]:
leftContexts = []
rightContexts = []

for seed in seeds:
    lcPerSeed = []
    rcPerSeed = []
    for m in re.finditer(seed, ecommerceData):
        start = m.start()
        end   = m.end()
        lcPerSeed.append(ecommerceData[:start].strip()[::-1])
        rcPerSeed.append(ecommerceData[end:].strip())
    lcPerSeed = getAllPrefixes(lcPerSeed)
    rcPerSeed = getAllPrefixes(rcPerSeed)
    leftContexts.append(lcPerSeed)
    rightContexts.append(rcPerSeed)

allLeftContexts = list(set([item for sublist in leftContexts for item in sublist]))
allRightContexts = list(set([item for sublist in rightContexts for item in sublist]))
print("All left contexts are ")
print(len(allLeftContexts))
# a11 = getAllPrefixes(allLeftContexts)
a1 = []
for item in a11:
    a1.append(item[::-1])
b1 = getAllPrefixes(allRightContexts)




print("All left patterns")
print(a1)
print("All right patterns")
print(b1)
# results = []
# for lPattern in a1:
#     leftPatternLength = len(lPattern)
#     if leftPatternLength<=0:
#         continue
#     for rPattern in b1:
#         rightPatternLength = len(rPattern)
#         if rightPatternLength<=0:
#             continue
#         pos = 0
#         wholePage = ecommerceData
#         while True:
#             pos=wholePage.find(lPattern)
#             if pos==-1:
#                 break
#             wholePage = wholePage[pos+leftPatternLength:]
#             pos = wholePage.find(rPattern)
#             if(pos==-1):
#                 break
#             result = wholePage[:pos].strip()
#             results.append(result)
# #             if (result.find(lPattern) == -1) and (result.find(rPattern)==-1):
# #                 results.append(result)

#             wholePage = wholePage[pos+rightPatternLength:]
# print("Results are ")
# for item in results:
#     print(item)


All left contexts are 
4


UnboundLocalError: local variable 'i' referenced before assignment

In [13]:
import os
ecommerceFile = "data/Lenovo Vibe K5.html"
ecommerceData = ""
with open(ecommerceFile) as f:
    ecommerceData = f.read()

presentSeeds = []
seedStartPosition = []
seedEndPosition  = []

for seed in seeds:
    seedL

for seed in seeds:
    seedLocInPage = ecommerceData.find(seed)
    seedLength = len(seed)
    seedEndLoc  = seedLocInPage + seedLength
    if seedLocInPage==-1:
        continue
    else:
        presentSeeds.append(seed)
        seedStartPosition.append(seedLocInPage)
        seedEndPosition.append(seedEndLoc)
        
print("Present seeds are: ")
print(presentSeeds)

leftContexts = []
rightContexts = []

for seed in 





for pos in seedStartPosition:
    leftContexts.append(ecommerceData[:pos].strip()[::-1])
    
leftPattern = os.path.commonprefix(leftContexts)[::-1]
leftPatternLength = len(leftPattern)
rightContexts = []
print("Left pattern is ")
print(leftPattern)
for pos in seedEndPosition:
    rightContexts.append(ecommerceData[pos:].strip())
rightPattern = os.path.commonprefix(rightContexts)
rightPatternLength  = len(rightPattern)

print("rihgt pattern is ")
print(rightPattern)
wholePage = ecommerceData
results = []
pos = 0

while True:
    pos=wholePage.find(leftPattern)
#     print("pos is " + str(pos))
    if pos==-1:
        break
    wholePage = wholePage[pos+leftPatternLength:]
    pos = wholePage.find(rightPattern)
    if(pos==-1):
        break
    results.append(wholePage[:pos].strip())
    wholePage = wholePage[pos+rightPatternLength:]
print("results are")
print(results)

SyntaxError: invalid syntax (<ipython-input-13-fccaff1d8591>, line 31)

In [ ]:
#get left and right patterns
leftContexts = []
for seed in presentSeeds:
    seedLocInPage = ecommerce